- [Linear Regression](#linear-regression)
- [Logistic Regression](#logistic-regression)

In [1]:
import probpy as pp
import numpy as np
import numba
import random

linear regression
---

In [2]:
%%time
def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]

w = [5, -2, 1, 5] # True underlying model
x = np.random.rand(100, 3) * 10
y = predict(w, x) + pp.normal.sample(mu=0, sigma=1, size=100).reshape(-1)



prior = pp.multivariate_normal.med(mu=np.ones(4) * 0, sigma=np.eye(4) * 5)
likelihood = pp.unilinear.med(sigma=1) # There exist an implementation for linear because it has a conjugate prior

for i in range(100):
    data = (y[i], x[i])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, priors=prior)
    
    if i % 10 == 0:
        w_approx = pp.mode(prior)
        print("Parameter Estimate", w_approx)
        
        print("Prior MSE", np.square(y - predict(w_approx, x)).mean(), 
              "True MSE", np.square(y - predict(w, x)).mean())
        print()

Parameter Estimate [-0.0160117  -0.62383897 -0.36849228 -0.06828899]
Prior MSE 1142.652826541197 True MSE 1.1587288019495965

Parameter Estimate [ 4.99997534 -1.87685696  0.9016167   4.81604376]
Prior MSE 1.3239918251783362 True MSE 1.1587288019495965

Parameter Estimate [ 4.98462374 -1.94817596  0.89549186  5.23407861]
Prior MSE 1.215893897492092 True MSE 1.1587288019495965

Parameter Estimate [ 4.9541552  -1.99959711  0.95003274  5.38160818]
Prior MSE 1.1393224077407944 True MSE 1.1587288019495965

Parameter Estimate [ 4.93644875 -1.98757253  0.97640099  5.29523886]
Prior MSE 1.1442630866261825 True MSE 1.1587288019495965

Parameter Estimate [ 4.94457836 -1.98230951  0.96725416  5.34798039]
Prior MSE 1.1330149119534174 True MSE 1.1587288019495965

Parameter Estimate [ 4.95935841 -1.98072199  0.9726215   5.27127344]
Prior MSE 1.1328066490778828 True MSE 1.1587288019495965

Parameter Estimate [ 4.95383092 -1.99215765  0.97646527  5.35034165]
Prior MSE 1.1327604188326323 True MSE 1.1587

logistic regression
---

In [3]:
%%time
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior

def likelihood(y, x, w): # This should be as fast as possible since it is the bottle-neck
    # It is possible to rewrite this loop in pure numpy with some broadcasting magic but 
    # I think this example is more instructive
        
    result = []
    for _w in w:
        prediction = sigmoid(predict(_w, x))
        result.append(pp.normal.p(y - prediction, mu=0, sigma=1.0))
        
    return result

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 5)

for i in range(50):
    j = random.randint(0, 80)
    data = (y[j: j + 20], x[j: j + 20])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   priors=prior, 
                                   match_moments_for=pp.multivariate_normal,
                                   batch=50,
                                   samples=5000,
                                   burn_in=500, 
                                   energies=1.0)

    if i % 10 == 0:
        w_approx = pp.mode(prior)
        print("Parameter Estimate", w_approx)
        
        print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
              "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
        print()

Parameter Estimate [-0.23601055  0.05588377  0.34628096 -0.71836587]
Prior MSE 0.0995996134116718 True MSE 0.031297186525624035

Parameter Estimate [-2.04545956  1.82695563  3.30731125 -1.82869796]
Prior MSE 0.032569237871609785 True MSE 0.031297186525624035

Parameter Estimate [-2.47703965  2.42116223  3.879914   -2.25933299]
Prior MSE 0.030886236193817557 True MSE 0.031297186525624035

Parameter Estimate [-2.76113245  2.88005064  4.22459557 -2.48620847]
Prior MSE 0.03170789237288279 True MSE 0.031297186525624035

Parameter Estimate [-3.35884969  3.13486393  4.94900395 -2.85277206]
Prior MSE 0.03205663697020225 True MSE 0.031297186525624035

CPU times: user 1min 33s, sys: 2min 24s, total: 3min 57s
Wall time: 60 s


Logistic Regression + Numpy magic ~ 10x speed-up
---

In [8]:
%%time
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior

def likelihood(y, x, w):
    return pp.normal.p((y - sigmoid(x @ w[:, :-1, None] + w[:, None, None, -1]).squeeze(axis=2)),
                    mu=0.0, sigma=1.0)
    
    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 5)

for i in range(50):
    j = random.randint(0, 80)
    data = (y[j: j + 20], x[j: j + 20])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   priors=prior, 
                                   match_moments_for=pp.multivariate_normal,
                                   batch=50,
                                   samples=5000,
                                   mixing=500, 
                                   energies=1.0)

    if i % 10 == 0:
        w_approx = pp.mode(prior)
        print("Parameter Estimate", w_approx)
        
        print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
              "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
        print()

Parameter Estimate [-0.53601026  0.28077979  0.46164084 -0.8330969 ]
Prior MSE 0.0782992400216686 True MSE 0.027590952878289207

Parameter Estimate [-1.96945747  1.46968586  3.37183621 -1.93631875]
Prior MSE 0.02917472725253408 True MSE 0.027590952878289207

Parameter Estimate [-2.54491043  2.14892638  4.12655498 -2.41129613]
Prior MSE 0.026715544705368424 True MSE 0.027590952878289207

Parameter Estimate [-2.78140459  2.54598524  4.3280773  -2.57244346]
Prior MSE 0.0262718273798136 True MSE 0.027590952878289207

Parameter Estimate [-3.13342456  2.65961273  4.56699265 -2.7096384 ]
Prior MSE 0.026869735122740566 True MSE 0.027590952878289207

CPU times: user 14.8 s, sys: 22.8 s, total: 37.6 s
Wall time: 9.41 s


Logistic Regression without moment-matching
---

In [12]:
%%time
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior

def likelihood(y, x, w):
    return pp.normal.p((y - sigmoid(x @ w[:, :-1, None] + w[:, None, None, -1]).squeeze(axis=2)),
                    mu=0.0, sigma=1.0)
    
    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 5)

for i in range(50):
    j = random.randint(0, 80)
    data = (y[j: j + 20], x[j: j + 20])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   priors=prior, 
                                   batch=5,
                                   samples=1000,
                                   mixing=100, 
                                   energies=1.0)

    if i % 10 == 0:
        modes = pp.mode(prior) # modes are sorted in order first is largest

        print("Number of modes", len(modes))
        w_approx = modes[0]
        
        print("Parameter Estimate", w_approx)
        
        print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
              "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
        print()

Number of modes 5
Parameter Estimate [ 0.25289025  1.07358778  0.22271974 -1.45592379]
Prior MSE 0.08989462571191517 True MSE 0.02557124439040531

Number of modes 3
Parameter Estimate [-1.6441759   1.24321202  2.4961304  -1.67858393]
Prior MSE 0.03433557116653359 True MSE 0.02557124439040531

Number of modes 3
Parameter Estimate [-3.08648632  2.71105106  2.76933978 -1.44667161]
Prior MSE 0.030504307227619015 True MSE 0.02557124439040531

Number of modes 3
Parameter Estimate [-3.40101005  2.76046226  3.64808157 -1.80800666]
Prior MSE 0.027947714687514585 True MSE 0.02557124439040531

Number of modes 3
Parameter Estimate [-3.56976171  2.86013503  3.1125747  -1.88175391]
Prior MSE 0.031942999881937405 True MSE 0.02557124439040531

CPU times: user 14 s, sys: 747 ms, total: 14.8 s
Wall time: 13.9 s


Logistic Regression with other posterior estimate 
---

this one handle more samples better

In [3]:
%%time
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior

def likelihood(y, x, w):
    return pp.normal.p((y - sigmoid(x @ w[:, :-1, None] + w[:, None, None, -1]).squeeze(axis=2)),
                    mu=0.0, sigma=1.0)
    
    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 10)

for i in range(5):
    data = (y, x)
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   priors=prior, 
                                   batch=500,
                                   samples=50000,
                                   mixing=1000, 
                                   energies=0.2,
                                   classical_mcmc=False,
                                   normalize=False)

    modes = pp.mode(prior) # modes are sorted in order first is largest

    
    print("Number of modes", len(modes))
    for mode in modes:
        print(mode)

    w_approx = modes[0]

    print("Parameter Estimate", w_approx)

    print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
          "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
    print()

Number of modes 5
[-1.83495405  2.12437659  2.70731731 -1.78704936]
[-1.76889682  1.49287281  1.38067777 -0.80976584]
[-2.34342496  1.12824225  2.30478095 -0.95576975]
[-1.64706129  1.35274162  2.7299821  -0.83616465]
[-2.44541939  1.56851505  2.89067823 -1.06378902]
Parameter Estimate [-1.83495405  2.12437659  2.70731731 -1.78704936]
Prior MSE 0.026942293441391398 True MSE 0.023097401887404934

Number of modes 3
[-2.29305721  1.21876176  2.5607145  -1.24814252]
[-1.86779886  1.81675884  2.72816997 -1.37242347]
[-1.64436763  1.79514194  2.82251421 -1.76011234]
Parameter Estimate [-2.29305721  1.21876176  2.5607145  -1.24814252]
Prior MSE 0.03147098188330927 True MSE 0.023097401887404934

Number of modes 4
[-2.25928324  1.51489425  2.84315756 -1.70546191]
[-1.76364171  1.93377866  2.78418663 -1.67628172]
[-2.22871414  1.76638158  2.42025249 -1.34629755]
[-2.72347522  1.35603043  2.78120651 -1.35553109]
Parameter Estimate [-2.25928324  1.51489425  2.84315756 -1.70546191]
Prior MSE 0.0323

IRT 
---

In [ ]:

# TODO